# Gradient Boosted Decision Tree to Predict RUL - Smoothed Data

This notebook is an extension of workbook "GBDT-Raw.ipynb" and is intended to analyze smoothed data. Additional notes and highlights are included herein. 

Ultimately, the results of the smoothed data did not generalize between train and test sets, and further analysis was abandoned. 

# 1.0 Dependencies and Notes

This notebook was built with the libraries imported below and the following versions:

Pandas 2.2.3 <br>
Numpy 2.0.2 <br>
Altair 5.4.1 <br>
sklearn 1.5.0 <br>
XGBoost 2.1.2 <br>
matplotlib 3.9.0 <br>

Different versions of these libraries may affect the functionality of this notebook.

The purpose of this notebook is to create a gradiet-boosted random forest to predict remaining useful life of jet engines using <b>smoothed</b> data provided by NASA. The notebook includes definitions to build the model, fit it, and then explore and store the results. 

Results are stored via a CSV file. There is a function for looping through different parameters, and other functions for viewing, exploring, and saving the results. 

NOTE REGARDING SMOOTHED DATA ANALYSIS: This notebook was built with notebook "GBDT-Raw.ipynb" used as a template. "GBDT-Raw.ipynb" used unsmoothed data, while this one uses smoothed data to reduce undesired noise in the data set. 

In [34]:
import pandas as pd
import numpy as np
import altair as alt
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from numpy import array, hstack
import pickle
import xgboost as xg
import matplotlib
import matplotlib.pyplot as plt

In [35]:
print("IMPORTED VERSIONS OF DEPENDENT LIBRARIES:")
print(pd.__version__)
print(np.__version__)
print(alt.__version__)
print(sklearn.__version__)
print(xg.__version__)
print(matplotlib.__version__)

IMPORTED VERSIONS OF DEPENDENT LIBRARIES:
2.2.3
2.0.2
5.4.1
1.5.0
2.1.2
3.9.0


The cell below can be used to view active variables in the notebook. It can be referred back to as the user goes through the notebook. 

In [36]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
%whos

Variable                         Type                          Data/Info
------------------------------------------------------------------------
GradientBoostingRegressor        ABCMeta                       <class 'sklearn.ensemble.<...>adientBoostingRegressor'>
INDX                             Index                         Index([    0,     1,     <...>ype='int64', length=3000)
MinMaxScaler                     type                          <class 'sklearn.preproces<...>sing._data.MinMaxScaler'>
RUL_values_to_remove             ndarray                       30: 30 elems, type `int64`, 240 bytes
StandardScaler                   type                          <class 'sklearn.preproces<...>ng._data.StandardScaler'>
alt                              module                        <module 'altair' from 'C:<...>es\\altair\\__init__.py'>
array                            builtin_function_or_method    <built-in function array>
columns                          list                          n=15
c

## 1.1 Load and define smoothed train and test data. 

Since the format of data input into the GBDT does not match that of other algorithms (cheifly, the neural networks), additional data preprocessing is conducted below. It maintains the same dropped columns and 'early RUL' as provided in the preprocessing workbook. Scaling occurs later in Section 2.1 and is optional. No batching occurs, as data is required to be 2 dimensional input for the GBDT. 

NOTE REGARDING SMOOTH DATA ANALYSIS: Smooth data is not processed from the raw NASA data like in "GBDT-Raw.ipynb". Instead, it is cleaned and smoothed ahead of time. A standard scaler is applied to it as well. 

Variables to keep active include early_RUL, and data variables for further processing: smooth_train, smooth_test, y_test.
<ul>
    <li><b>early_RUL</b> - sets the threshold of the maximum remaining useful life. This is intended to lessen the impact of data instance far from the end of useful life; see the data processing notebook for more. </li>
    <li><b>raw_train, raw_test, y_test</b> - the raw, unprocessed data from the NASA data repository. The raw files are preprocessed in Section 1.1 of this notebook (and maintain the same variable names) to remove unneeded fields and to create the RUL column of the training set.</li>
<ul>

In [37]:
early_RUL = 125
link = "../data/processed_data_pickle_files_with_smoothing/"

with open(link + "train_data_no_batches.pkl", 'rb') as f:
    smooth_train = pickle.load(f)
    
del link

columns = ['unit_number', 'sensor_2', 'sensor_3', 'sensor_4',
       'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_11',
       'sensor_12', 'sensor_13', 'sensor_15', 'sensor_17', 'sensor_20',
       'sensor_21']

smooth_train.columns = columns
smooth_train['unit_number'] = smooth_train['unit_number'].astype('int')

smooth_train.insert(1, 'time_cycles', 1)

for u in smooth_train["unit_number"].unique():
    l = len(smooth_train[smooth_train['unit_number'] == u]) + 1
    smooth_train.loc[smooth_train['unit_number'] == u, 'time_cycles'] = list(range(1, l, 1))

print(len(smooth_train))

20631


The defined function and code below adds early RUL to the data and is borrowed from the notebook 'data_processing.ipynb'. 

In [38]:
def process_targets(data_length, early_rul=None):
    
    # if no early RUL is provided, generate a descending sequence from data_length -1 to 0
    if early_rul is None:
        return np.arange(data_length - 1, -1, -1)
    
    else:
        # calculate the duration for which early RUL is applicable
        early_rul_duration = data_length - early_rul
        
        # if the early RUL duration is non-positive, use a linear degradation curve (same as when early_rul is none)
        if early_rul_duration <= 0:
            return np.arange(data_length - 1, -1, -1)
        
        else:
            # create an array where the first early_rul_duration values are equal to early_rul
            target_array = np.append(early_rul * np.ones(shape=(early_rul_duration,)),
                                     np.arange(early_rul - 1, -1, -1))  # Add descending values from early_rul-1 to 0
            return target_array
        
smooth_y_train = np.ndarray([])

for engine in smooth_train.iloc[:,0].unique():
    
    data_length = len(smooth_train[smooth_train.iloc[:,0] == engine])
    if engine == 1:
        smooth_y_train = process_targets(data_length, early_rul = early_RUL)
    else:
        new = process_targets(data_length, early_rul = 125)
        smooth_y_train = np.append(smooth_y_train, new)

del process_targets
        
print(len(smooth_y_train))
print(smooth_y_train)

20631
[125. 125. 125. ...   2.   1.   0.]


Additional processing is required for the test data, since smoothing limited the test engine count to 70. This is presented below.

In [39]:
link = "../data/processed_data_pickle_files_with_smoothing/"

with open(link + "test_data_no_batches.pkl", 'rb') as f:
    smooth_test = pickle.load(f)
    
with open(link + "true_rul_values.pkl", 'rb') as f:
    smooth_y_test = pickle.load(f)
    
del link

columns = ['unit_number', 'sensor_2', 'sensor_3', 'sensor_4',
       'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_11',
       'sensor_12', 'sensor_13', 'sensor_15', 'sensor_17', 'sensor_20',
       'sensor_21']

smooth_test.columns = columns

interpolated_engines_to_remove = [1, 2, 5, 9, 11, 14, 15, 22, 25, 26, 
                                  33, 39, 44, 47, 48, 50, 59, 65, 67, 
                                  69, 71, 75, 78, 83, 85, 87, 88, 95, 96, 99]

RUL_values_to_remove = np.array(interpolated_engines_to_remove) - 1
smooth_y_test = np.delete(smooth_y_test, RUL_values_to_remove)

smooth_test['unit_number'] = smooth_test['unit_number'].astype('int')


INDX = smooth_test[smooth_test['unit_number'].isin(interpolated_engines_to_remove)].index

smooth_test.drop(INDX, inplace = True)

smooth_test.insert(1, 'time_cycles', 1)

for u in smooth_test["unit_number"].unique():
    l = len(smooth_test[smooth_test['unit_number'] == u]) + 1
    smooth_test.loc[smooth_test['unit_number'] == u, 'time_cycles'] = list(range(1, l, 1))
    
smooth_test.reset_index(drop = True, inplace = True)

In [40]:
print(smooth_train.shape)
print(smooth_y_train.shape)
print(smooth_test.shape)
print(smooth_y_test.shape)

(20631, 16)
(20631,)
(11097, 16)
(70,)


## 1.2 Capturing Headings and Choosing Features

This section is for reference and does not affect the raw data files used in the following sections. 

Hard coded descriptions of the different sensors are provided below, both abbreviated descriptions (to use as column headers if so desired) and longer descriptions that can be printed out. Dictionaries are created for each based on sensor number.

Then, the sensors that were removed are shown, along with the sensors that were kept in the dataset. 

Only variables for the dictionaries are kept: feature_dictionary_short, feature_dictionary_long

In [41]:
description_headers = ["Temp, fan in", "Temp, LPC out", "Temp, HPC out", 
                           "Temp, LPT out", "Press, fan in", "Tot Press, bypass", "Tot Press, HPC out",
                           "Speed, fan", "Speed, core", "Eng Press Ratio", "Stat Press, HPC out", 
                           "phi Fuel Flow Ratio", "Corr. Speed, Fan", "Corr. Speed, Core", "Bypass Ratio", 
                           "Burner Fuel/Air Ratio", "Bleed Enthalpy", "Dem Speed, fan", "(?) Dem Corr Speed, fan", 
                           "Coolant Bleed, HPT", "Coolant Bleed, LPT" ]
long_descriptions = ["Total temperature at fan inlet, degrees rankine", 
                     "Total temperature at Low Pressure Compressor (LPC) outlet, degrees rankine", 
                     "Total temperature at High Pressure Compressor (HPC) outlet, degrees rankine", 
                     "Total temperature at Low Pressure Turbine (LPT) outlet, degrees rankine", 
                     "Pressure at fan inlet, psia", 
                     "Total pressure in bypass-duct, psia", 
                     "Total pressure at HPC outlet, psia", 
                     "Physical fan speed, rpm", 
                     "Physical core speed, rpm", 
                     "Engine pressure ratio (P50/P2) where P2 is Pressure at Fan Inlet and P50 is Pressure at LPT outlet, psia", 
                     "Static pressure at HPC outlet, psia", 
                     "Ratio of fuel flow to Ps30 where Ps30 is static pressure at HPC outlet, pps/psi", 
                     "Corrected fan speed, rpm", 
                     "Corrected core speed, rpm", 
                     "Bypass Ratio, unitless", 
                     "Burner fuel-air ratio, unitless", 
                     "Bleed Enthalpy, unitless", 
                     "Demanded fan speed, rpm", 
                     "Demanded corrected fan speed, rpm", 
                     "HPT coolant bleed, lbm/s", 
                     "LPT coolant bleed, lbm/s"]
    #Temps are in R; for temps in F, subtract 459.67
    #Pressures are in psia
    #Speed is in rpm
    #"phi Fuel Flow Ratio" is ration of fuel flow to static pressure at HPC, in pps/psi
    #Bypass ratio - proportion of air mass passing through bypass versus the engine core (compressors/burners/turbines)
    #"Bleed Enthalpy" refers to bleed air (?), and the total enthalpy of it (Enthalpy = Internal Energy + (Pressure*Volume))
    #Coolant Bleed is in pound mass per second (lbm/s)
    
sensor_names = ['sensor_{}'.format(i) for i in range(1, 22)]

feature_dictionary_short = {}
feature_dictionary_long = {}
for i, sensor in enumerate(sensor_names):
    feature_dictionary_short[sensor] = description_headers[i]
    feature_dictionary_long[sensor] = long_descriptions[i]
    
for key in feature_dictionary_long.keys():
    print(key, ": ", feature_dictionary_long[key])
    
del description_headers
del long_descriptions
del sensor_names

smooth_train.head()

sensor_1 :  Total temperature at fan inlet, degrees rankine
sensor_2 :  Total temperature at Low Pressure Compressor (LPC) outlet, degrees rankine
sensor_3 :  Total temperature at High Pressure Compressor (HPC) outlet, degrees rankine
sensor_4 :  Total temperature at Low Pressure Turbine (LPT) outlet, degrees rankine
sensor_5 :  Pressure at fan inlet, psia
sensor_6 :  Total pressure in bypass-duct, psia
sensor_7 :  Total pressure at HPC outlet, psia
sensor_8 :  Physical fan speed, rpm
sensor_9 :  Physical core speed, rpm
sensor_10 :  Engine pressure ratio (P50/P2) where P2 is Pressure at Fan Inlet and P50 is Pressure at LPT outlet, psia
sensor_11 :  Static pressure at HPC outlet, psia
sensor_12 :  Ratio of fuel flow to Ps30 where Ps30 is static pressure at HPC outlet, pps/psi
sensor_13 :  Corrected fan speed, rpm
sensor_14 :  Corrected core speed, rpm
sensor_15 :  Bypass Ratio, unitless
sensor_16 :  Burner fuel-air ratio, unitless
sensor_17 :  Bleed Enthalpy, unitless
sensor_18 :  Dema

,unit_number,time_cycles,sensor_2,sensor_3,sensor_4,sensor_6,sensor_7,sensor_8,sensor_9,sensor_11,sensor_12,sensor_13,sensor_15,sensor_17,sensor_20,sensor_21
0,1,1,-1.065889,-0.695171,-0.807599,0.426686,1.004786,-0.570566,-0.751606,-0.398120,1.288601,-0.822016,-1.038086,-1.154212,1.307804,0.969818
1,1,2,-1.100972,-0.665510,-0.865381,0.426686,1.005885,-0.578016,-0.752771,-0.556099,1.196665,-0.829922,-1.037966,-1.158125,1.276253,0.989448
2,1,3,-1.102540,-0.647767,-0.902253,0.426686,1.002206,-0.571604,-0.751847,-0.705007,1.159946,-0.829644,-1.044008,-1.148715,1.239163,1.011182
3,1,4,-1.097768,-0.642849,-0.936744,0.426686,1.001796,-0.561942,-0.748910,-0.835521,1.111974,-0.825451,-1.046743,-1.137596,1.206491,1.037151
4,1,5,-1.082796,-0.648783,-0.960949,0.426686,1.003173,-0.555888,-0.743812,-0.928732,1.059255,-0.820618,-1.040194,-1.127616,1.178001,1.057445


In [42]:
columns_to_drop = []
for key in feature_dictionary_long.keys():
    if key not in smooth_train.iloc[:,2:].keys():
        columns_to_drop.append(key)

print('SENSORS REMOVED') 
for item in columns_to_drop:
    print(item + " : ", feature_dictionary_long[item])

SENSORS REMOVED
sensor_1 :  Total temperature at fan inlet, degrees rankine
sensor_5 :  Pressure at fan inlet, psia
sensor_10 :  Engine pressure ratio (P50/P2) where P2 is Pressure at Fan Inlet and P50 is Pressure at LPT outlet, psia
sensor_14 :  Corrected core speed, rpm
sensor_16 :  Burner fuel-air ratio, unitless
sensor_18 :  Demanded fan speed, rpm
sensor_19 :  Demanded corrected fan speed, rpm


In [43]:
print('SENSORS KEPT AS FIELDS') 
for key in feature_dictionary_long.keys():
    if key in smooth_train.iloc[:,2:].keys():
        print(key + " : ", feature_dictionary_long[key])

SENSORS KEPT AS FIELDS
sensor_2 :  Total temperature at Low Pressure Compressor (LPC) outlet, degrees rankine
sensor_3 :  Total temperature at High Pressure Compressor (HPC) outlet, degrees rankine
sensor_4 :  Total temperature at Low Pressure Turbine (LPT) outlet, degrees rankine
sensor_6 :  Total pressure in bypass-duct, psia
sensor_7 :  Total pressure at HPC outlet, psia
sensor_8 :  Physical fan speed, rpm
sensor_9 :  Physical core speed, rpm
sensor_11 :  Static pressure at HPC outlet, psia
sensor_12 :  Ratio of fuel flow to Ps30 where Ps30 is static pressure at HPC outlet, pps/psi
sensor_13 :  Corrected fan speed, rpm
sensor_15 :  Bypass Ratio, unitless
sensor_17 :  Bleed Enthalpy, unitless
sensor_20 :  HPT coolant bleed, lbm/s
sensor_21 :  LPT coolant bleed, lbm/s


# 2.0 Definitions

Definitions used in the model creation and tuning. 

## 2.1 Function Definitions for Data Processing

Data can be scaled, but may not be neccessary with decision trees.

Two options for a scaler exist - 'standard' and 'minmax', both built off the canned sklearn scalers. 

Since decision trees consider each instance separately (as a singular moment in time), extra consideration was given to how the different time series fields are changing over the time cycles. To do this, the below functions break apart the data set into the separate engines, add additional fields representing changes in each time series field, and then rebuild the dataset by concatenating the engines back into a single dataframe with the new fields (this is Step 4 in the main pipeline function defined below). 

For each existing field, one or two additional fields are possible to represent the observed changes in that field. The changes are measured using windows and are summarized as follows:

<ul>
  <li>Average Change ('avg'): average change between each consecutive time step within the designated window. The larger the time window, the larger the time frame the field considers for average change.</li><br>
  <li>Absolute Change ('abs'): absolute change between the first and last rows in a window.</li><br>
  <li>Acceleration of Change ('acc'): Measures if the rate of change of the field is increasing or decreasing, and to what extent. It does this by first taking the difference as defined below with input 'dif', and then taking the absolute differences (dependent on window size) of those differences.</li><br>
    <li>Difference ('dif'): The difference between the current instance and the immediate preceding instance. Is the same as absolute change with a window size of 2. </li>
</ul>
This methodoly creates some fields with NaN values at the beginnning of each engine's time series. An option is available to drop the rows with NaN values with the default option being "True" (default is to drop NaN values). 

In [44]:
def add_window_field(data, size, typ, min_p, num):
    if num == "1":
        cols = list(data.columns)
    elif num == "2":
        cols = list(data.columns[0:14])
    
    for col in cols:
        dif = data[col].diff()
        name = "w" + num + "_" + typ + "_" + col
        lamb_x = lambda x: x.iloc[-1] - x.iloc[0]
        if typ == 'avg':
            data[name] = dif.rolling(window = size, min_periods = min_p).mean()
        elif typ == 'abs':
            data[name] = data[col].rolling(window = size, min_periods = min_p).apply(lamb_x)
        elif typ == 'acc':
            data[name] = dif.rolling(window = size, min_periods = min_p).apply(lamb_x)
        elif typ == 'dif':
            data[name] = dif
        else:
            raise NameError("The window type was not properly specified.")           
    return data

def add_window_fields(dat, p, num, show_prints = False):
    #Initialize new dataframe, exclude identifier columns (unit and cycle)
    n_tn = dat.copy().iloc[:0,2:]
    
    #Break apart existing dataframe by engine
    for unit in dat['unit_number'].unique():
            #Create new fields w/ function
            chunk = dat[dat['unit_number'] == unit].iloc[:,2:]
            if show_prints == True:
                print("Unit ", unit)
                display(chunk)    
            chunk = add_window_field(chunk, 
                                     p["first_window"], 
                                     p["first_window_type"], 
                                     p["min_periods"], 
                                     num)
            if show_prints == True:
                display(chunk)
            #Append chunk to newly initialized dataframe
            n_tn = pd.concat([n_tn, chunk], ignore_index = True)
            
    #First add back in identifier columns, then return new dataframe
    n_tn = pd.concat([dat.iloc[:,:2], n_tn], axis = 1, ignore_index = False)
    return n_tn

def process_data(params, train, test, y_train):
    
    #STEP 1
    #Throw an error if the designated windows are too big. 
    if params['first_window'] != None:
        if params['first_window'] > 30:
            raise ValueError('Window size cannot be greater than 30 to maintain functionality with the given test set.')
    if params['second_window'] != None:
        if params['second_window'] > 30:
            raise ValueError('Window size cannot be greater than 30 to maintain functionality with the given test set.')
    
    #STEP 2
    #Initialize required variables/datasets used in this function definition.
    trn = train.iloc[:,2:]
    tst = test.iloc[:,2:]
    y_trn = y_train
    cols = train.columns[2:]
    before = len(trn)
    
#     print("STEP 2:")
#     display(tst.head(5))
    
    #STEP 3
    #Scale the test and train sets fit on all the fields in the train set (regardless of unit).
    #Don't forget to reattach the 'unit_number' and 'cycles'
    if params['scaler'] == 'standard':
        scaler = StandardScaler()
    elif params['scaler'] == 'minmax':
        scaler = MinMaxScaler()
    elif params['scaler'] == None:
        scaler = None
    else:
        raise NameError("The designated scaler does not exist or is not available.")
    
    if scaler != None:
        trn = pd.DataFrame(scaler.fit_transform(trn), columns = cols)
        tst = pd.DataFrame(scaler.transform(tst), columns = cols)

    trn = pd.concat([train.iloc[:,:2], trn], axis = 1)
    tst = pd.concat([test.iloc[:,:2], tst], axis = 1)
    
#     print("STEP 3:")
#     display(tst.head(5))
    
    #STEP 4 (OPTIONAL BOOLEAN - USES PREVIOUSLY DEFINED FUNCTIONS)
    #Add new features to test and train sets, keep it confined to individual engines.
    #Use if statement to determine if new columns are desired
    if params["first_window"] != None:
        trn = add_window_fields(trn, params, "1")
        tst = add_window_fields(tst, params, "1")
    if params["second_window"] != None:
        trn = add_window_fields(trn, params, "2")
        tst = add_window_fields(tst, params, "2")

#     print("STEP 4:")
#     display(tst.head(5))
    
    #STEP 5 (OPTIONAL BOOLEAN)
    #Drop na values from train and test sets, if desired. 
    if params['drop_nan_train'] == True:
        trn["RUL"] = y_trn
        trn.dropna(inplace = True)
        y_trn = trn["RUL"]
        trn.drop(['RUL'], axis = 1, inplace = True)
        
        tst.dropna(inplace = True)

#     print("STEP 5:")
#     display(tst.head(5))
    
    #STEP 6
    #Extract final row from each test set. 
    temp_df = tst.iloc[:0,:]
    for unit in tst['unit_number'].unique():
        tail = tst[tst['unit_number'] == unit].tail(1)
        temp_df = pd.concat([temp_df, tail], ignore_index = True)
    tst = temp_df
    
#     print("STEP 6:")
#     display(tst.head(5))
    
    #STEP 7
    #Record number of dropped rows from train data and number of features, to include in record.
    dropped = before - len(trn)
    num_feat = len(trn.columns)
    
    #STEP 7
    #Return each processed dataframe
    return trn, tst, y_trn, {'num_features': num_feat - 2, 'train_samples_dropped': dropped}

## 2.2 Model Function Definitions

In [45]:
def make_and_train_GBDT(params, train_data, train_target):
    
    """
    Intializes a gradient-boosted decision tree using sklearn Library. 
    Parmeters are entered as a library. 
    
    Inputs:
    params: library of input parameters, must include .... 
    
    """

    model = GradientBoostingRegressor(loss = 'squared_error', 
                                      learning_rate = params['mod_learning_rate'], 
                                      n_estimators = params['mod_n_estimators'], 
                                      subsample = params['mod_subsample'],   #Set to less than 1 to help with overfitting
                                      min_samples_split = params['mod_min_samples_split'], #Min to split into two branches
                                      min_samples_leaf = params['mod_min_samples_leaf'], #Min in each branch at a split-can help with overfitting
                                      max_depth = params['mod_max_depth'], 
                                      validation_fraction = 0.1, 
                                      n_iter_no_change = params['mod_validation']) 
    
    model.fit(train_data, train_target)
    
    return model

In [46]:
def GBDT_run_for_log(model, train_data, train_target, test_data, test_target, params):
    
    """ 
    Trains an input model on the input train data, then collects various scoring metrics of both the 
    train and test data. The input parameters dictionary is then concatenated with the metrics to provide 
    a dictionary of both the metrics and input parameters used. 
    
    Inputs:
    
    Model: Gradient-Boosted Decision Tree model from the previous function above ('make_and_train_GBDT') 
    or GBDT defined via other means
    
    Various data inputs: Train and Test, plus targets
    
    params: library of parameters. Must include 'perform_validation' and '#_epochs'.
    
    Output: library of train and test parameters, along with parameters included in the 'params' input. 
    
    """
    
    logger = {}
    
    y_hat_train = model.predict(train_data)
    y_hat_test = model.predict(test_data)
    
    logger["train_MSE"] = mean_squared_error(train_target, y_hat_train).item()
    logger["test_MSE"] = mean_squared_error(test_target, y_hat_test).item()
    
    logger["train_RMSE"] = np.sqrt(mean_squared_error(train_target, y_hat_train)).item()
    logger["test_RMSE"] = np.sqrt(mean_squared_error(test_target, y_hat_test)).item()
    
    logger["train_MAE"] = mean_absolute_error(train_target, y_hat_train).item()
    logger["test_MAE"] = mean_absolute_error(test_target, y_hat_test).item()
    
    logger["train_MAPE"] = np.mean(np.abs((train_target - y_hat_train) / y_hat_train)).item() * 100
    logger["test_MAPE"] = np.mean(np.abs((test_target - y_hat_test) / y_hat_test)).item() * 100
    
    logger["train_R2"] = r2_score(train_target, y_hat_train)
    logger["test_R2"] = r2_score(test_target, y_hat_test)
    
    logger.update(params)
    
    return logger
    
    

In [47]:
def add_to_logger(new_instance, existing_dict):
    
    """
    Takes in a new instance of the function 'GBDT_run_for_log' which returns the performance metrics 
    for a GBDT using the listed input parameters. It then adds that instance to a dictionary of lists,
    where each index in the list represents a new run of 'GBDT_run_for_log'. This is intended to be used
    in running loops during parameter tuning to keep track of which parameters perform the best. 
    
    Input:
    new_instance: a dictionary of the most recent parameters and performance metrics
    existing_dict: a dictionary of lists that keep a record of performance metrics and the parameters that
            led to those results
    
    Output:
    An updated record of parameters/performance metrics in which the most recent parameters are added to the record
    
    """
    
    if existing_dict == None:
        record = {}
        for key in new_instance.keys():
            record[key] = []
    else:
        record = existing_dict.copy()
        
    for key in record.keys():
        l = record[key]
        if key in new_instance.keys():
            l.append(new_instance[key])     
        else:
            l.append(np.nan)
        record[key] = l
        
    return record

In [48]:
def loop_through_parameters(loops, 
                            parameters, 
                            train, 
                            y_train, 
                            test, 
                            y_test):
    """
    Runs parameter loops for model and records the resulting metrics. Returns a dictionary that is 
    a log of the results, where each key represents a parameter or performance metric, and each item is
    a list where the indexes represent the runs in chronological order. 
    
    Inputs:
    loops: a dictionary of the the keys and values to loop through.
    params: a dictionary with all the parameters neccessary to build the CNN, train it, and acquire the 
    results using the functions defined previously.
    
    Output:
    A dictionary of lists with input parameters and resultant performance metrics. Can easily be used
    to construct a dataframe. 
    """
    
    record = None
    keys = list(loops.keys())
    
    def nested_function(p1 = None, p2 = None, p3 = None, p4 = None):
        parameters[keys[0]] = p1
        if len(keys) > 1:
            parameters[keys[1]] = p2
        if len(keys) > 2:
            parameters[keys[2]] = p3
        if len(keys) > 3:
            parameters[keys[3]] = p4
            
#         print(parameters)
#         print(test.columns)

        trn, tst, trn_tar, d = process_data(parameters, train, test, y_train)

        trn = trn.iloc[:,2:]
        tst = tst.iloc[:,2:]
#         print(trn.shape)
#         print(tst.shape)
#         print(trn_tar.shape)
#         print(y_test)
        
        model = make_and_train_GBDT(parameters, trn, trn_tar)
        
        new_instance = GBDT_run_for_log(model, trn, trn_tar, tst, y_test, parameters)
        
        for key in d.keys():
            new_instance[key] = d[key]
        
        r = add_to_logger(new_instance, record)
        
        return r
        
    if len(loops.keys()) == 1:
        for p_1 in loops[keys[0]]:
            print("Starting first loop with value ", p_1, ".")
            record = nested_function(p1 = p_1)
            
    elif len(loops.keys()) == 2:
        for p_1 in loops[keys[0]]:
            print("Starting first loop with value ", p_1, ".")
            for p_2 in loops[keys[1]]:
                record = nested_function(p1 = p_1, p2 = p_2)
                
    elif len(loops.keys()) == 3:
        for p_1 in loops[keys[0]]:
            print("Starting first loop with value ", p_1, ".")
            for p_2 in loops[keys[1]]:
                for p_3 in loops[keys[2]]:
                    record = nested_function(p1 = p_1, p2 = p_2, p3 = p_3)
                    
    else:
        for p_1 in loops[keys[0]]:
            print("Starting first loop with value ", p_1, ".")
            for p_2 in loops[keys[1]]:
                for p_3 in loops[keys[2]]:
                    for p_4 in loops[keys[3]]:
                        print("     Running with variables ", p_2, ", ", p_3, ", and ", p_4, ".")
                        record = nested_function(p1 = p_1, p2 = p_2, p3 = p_3, p4 = p_4)
                        
    return record
        

In [49]:
def add_to_GBDT_log(record, link = 'GBDT_log.csv', save_changes = False):
    
    """
    Combines most recent group of models with those saved in the log file. Includes designating a
    tuning group based on the most recent tuning group in the log file.
    
    Inputs:
    record: most recent record set of parameter tunings, as returned by above functions.
    link: pathway and filename for the saved log file, it if exists. If it doesn't exist, this 
        function won't work.
    save_changes: designates whether to save the updates to the CSV file that stores the model parameter tuning results.
    
    Output:
    A dataframe of the combines records on file and the most recent turning group. 
    """
    
    df = pd.read_csv(link)
    group_num = df['tuning_group'].max() + 1
    r = pd.DataFrame(record)
    r.insert(0, 'tuning_group', group_num)
    
    df = pd.concat([df, r], ignore_index = True)

    if save_changes == True:
        df.to_csv(link, index = False)
    
    return df

In [50]:
def plot_results(data, fields):
    
    """
    Makes a simple Altair Chart for compairing results visually.
    
    Input:
    data: Dataframe that includes the fields from the most current record or from the CNN_log saved as a CSV.
    fields: designated fields to encode using shape and color. Default are the first two designated in the 
        most recent record.
        
    Output:
    A chart comparing changes in the designated fields, mapped against test RMSE and difference between
        train and test RMSE.
    """

    data['RMSE_diff'] = data['test_RMSE'] - data['train_RMSE']
    fields_to_keep = ["test_RMSE", "train_RMSE", "RMSE_diff"] + fields
    data = data[fields_to_keep]
    
    if len(fields) > 1:
        chart = alt.Chart(data).mark_point().encode(x = alt.X("test_RMSE").scale(zero = False), 
                                                y = 'RMSE_diff', 
                                                color = fields[0] + ":N", 
                                                shape = fields[1] + ":N")
    else:
        chart = alt.Chart(data).mark_point().encode(x = alt.X("test_RMSE").scale(zero = False), 
                                                y = 'RMSE_diff', 
                                                shape = fields[0] + ":N",
                                                color = fields[0] + ":N"
                                                )

    return chart

def show_df_of_results(data, fields):
    data['RMSE_diff'] = data['test_RMSE'] - data['train_RMSE']
    fields_to_keep = ["test_RMSE", "train_RMSE", "RMSE_diff"] + fields
    data = data[fields_to_keep]
    return data

# 3.0 Building Models and Exploring Results

Different processing procedures (for example, different window sizes and additional fields added) and model input parameters were used to explore the best data input variation and the best model parameter selection. The functions defined in Sections 2.1 and 2.2 are utilized to perform looping of different variations of model and data architecture. Results are stored in a .CSV file. 

## 3.1 Define the default parameters and those to change while looping. 

The cell below was used while tuning the GBDT model. Different "tuning groups" were used in loops and then investigated by studying plots and dataframes. Thereafter, a new tuning group would be created for further investigation. The goal was to find the best model that maximized the accuracy but minimized overtraining. 

The dictionary variable "parameters" hold default parameters used by the processing/model. The dictionary variable "loops" shows the fields that are looped through for each tuning group. If a field is not included in "loops", then the default value is used for each model iteration. 

For future reference, each tuning group was commented out after the results were recorded. 

NOTE FOR SMOOTHED DATA ANALYSIS: Only one tuning group was performed for the smoothed data. The models did not generalize; it was assumed this was due to an issue with the smoothing, and further analysis was abandoned. 

In [51]:
#Parameter List
parameters = {'scaler': None,  #Left as none because data is already scaled
            'first_window': None, 
            'first_window_type': 'abs', #options: avg, dif, acc, abs
            'second_window': None, 
            'second_window_type': 'avg', 
            'drop_nan_train': True,
            'min_periods': None, 
            'mod_loss': 'squared_error', 
            'mod_learning_rate': 0.2, 
            'mod_n_estimators': 400, 
            'mod_subsample': 1.0,
            'mod_min_samples_split': 2, 
            'mod_min_samples_leaf': 1, 
            'mod_max_depth': 3, 
            'mod_validation': None  #requires integer as input - number of iterations with no change
            }

loops = {
         'mod_learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5], #GROUP 1 - CHOOSE AROUND 0.4
         'mod_n_estimators': [10, 20, 50, 100, 200, 300, 500], #GROUP 1 - CHOOSE B/T 20 AND 100
         'mod_max_depth': [3, 5, 7, 9], #GROUP 1 - CHOOSE 2-5
#          'scaler': ['standard', 'minmax', None], #GROUP 1 - MAKES NO DIFFERENCE, STAY WITH STANDARD
#          'mod_learning_rate': [3.75, 0.4, 0.425, 0.45, 0.475], #GROUP 2 - CHOOSE 0.425
#          'mod_n_estimators': [15, 20, 25, 30, 35, 40, 45, 50, 55, 60], #GROUP 2 - CHOOSE 55
#          'mod_max_depth': [2, 3, 4], #GROUP 2 - CHOOSE 3
#          'mod_validation': [None, 2], #GROUP 2 - GO W/ NONE
#          'first_window': [3, 5, 7, 10, 15, 20], #GROUP 3 - try higher window sizes
#          'first_window_type': ['avg', 'dif', 'acc', 'abs'], #GROUP 3 - stick to avg and abs
#          'mod_n_estimators': [50, 55, 60, 70, 80, 100], #GROUP 3 - try higher number of estimators
#          'mod_learning_rate': [0.1, 0.425], #GROUP 3 - try wider range
#          'first_window': [12, 18, 20, 25, 30], #GROUP 4 - go with 30
#          'first_window_type': ['avg', 'abs'], #GROUP 4 - GO WITH avg
#          'mod_n_estimators': [100, 200, 300], #GROUP 4 - try 200 - 600
#          'mod_learning_rate': [0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5] #GROUP 4 - TRY .18 TO .32
#          'first_window': [20, 22, 24, 26, 28, 30], #GROUP 5 - KEEP 30
#          'first_window_type': ['avg', 'abs', 'acc'], #GROUP 5 - KEEP ABS
#          'mod_n_estimators': [250, 300, 350, 400, 450, 500], #GROUP 5 - KEEP 400
#          'mod_learning_rate': [0.175, 0.2, 0.225, 0.25, 0.275, 0.3, 0.325] #GROUP 5 - KEEP 0.2, BEST PROBABLY B/T .2 AND .25
#          'second_window': [3, 5, 10, 20], #GROUP 6
#          'second_window_type': ['avg', 'abs', 'acc', 'dif'], #GROUP 6
#          'mod_n_estimators': [300, 400, 500, 600], #GROUP 6
        }

## 3.2 Run processing/model loops and study results. 

Each tuning group was assessed manually and individually. The results were used to determine the looping parameters to use in the consecutive tuning group. Results are stored in the .CSV file. 

### 3.2.1 Run processing/model loops as defined in Section 3.1

The loops are ran with in the block below using the inputs designated in the block above.

The function below is commented out when not in use. 

In [52]:
#for each key and associated list in 'loops', make a record of results for different parameters.

# record = loop_through_parameters(loops, parameters, smooth_train, smooth_y_train, smooth_test, smooth_y_test)

### 3.2.2 Compare results graphically and in a dataframe

COMMENT FOR SMOOTHED DATA ANALYSIS: The below sets the log file equal to 'record' so it can be visualized.

In [53]:
record = pd.read_csv("GBDT_SMOOTH_Data_Log.csv")

Results from the most recent tuning group were viewed below. 

COMMENT FOR SMOOTHED DATA ANALYSIS: As can be seen below, the RMSE is significantly worse than for the unsmoothed data, while the difference in RMSE between the test and train set is not significantly less than 3 (poor generalization). This is better results with the smoothed data than with the CNN model, however not considered promising, so further analysis was abandoned. 

In [54]:
print(loops.keys())
plot_results(pd.DataFrame(record), list(loops.keys()))

display(plot_results(pd.DataFrame(record), ['mod_max_depth']))
display(plot_results(pd.DataFrame(record), ['mod_n_estimators']))
display(plot_results(pd.DataFrame(record), ['mod_learning_rate']))

dict_keys(['mod_learning_rate', 'mod_n_estimators', 'mod_max_depth'])


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

A table of the most recent results was viewed below, sorted by test RMSE or train/test performance difference. Only the fields that changed with each iterations are included.

In [55]:
df = show_df_of_results(pd.DataFrame(record), list(loops.keys()))
# df = df[abs(df["RMSE_diff"]) < 0.5].sort_values("RMSE_diff")
df = df.sort_values("RMSE_diff")[0:10]
# df.sort_values("test_RMSE")[0:10]
df


,test_RMSE,train_RMSE,RMSE_diff,mod_learning_rate,mod_n_estimators,mod_max_depth
29,30.784300,27.986273,2.798027,0.05,10,5
30,30.278209,27.467333,2.810876,0.05,10,7
10,30.499026,27.666356,2.832670,0.01,50,7
9,31.073727,28.177092,2.896635,0.01,50,5
8,32.033405,28.828552,3.204854,0.01,50,3
28,31.944122,28.632396,3.311726,0.05,10,3
6,38.518095,35.106675,3.411420,0.01,20,7
5,38.767308,35.300600,3.466708,0.01,20,5
7,38.385010,34.861057,3.523953,0.01,20,9
11,30.567546,27.026610,3.540936,0.01,50,9


The cell below was used to <b>initialize</b> the performance log CSV. This cell should not be uncommented unless the user wants to initialize a new log file for looping. 

In [56]:
#Initialize the log with the first tuning group. 
#FILENAME REMOVED - DO NOT OVERWRITE LOG!

# df = pd.DataFrame(record)

# df.insert(0, "tuning_group", 1)

# df.to_csv("GBDT_SMOOTH_Data_Log.csv", index = False)

The below block saves the most recent tuning group to the CSV log. The value for 'tuning_group' is determined based on the last entry into the log. 

In [57]:
# add_to_GBDT_log(record, save_changes = True)

The below block is used to explore the entire CSV log to compare results. Also organized by performace/overfitting to find the best parameters to test. 

In [58]:
df = pd.read_csv("GBDT_SMOOTH_Data_Log.csv")

##USE CTRL + "/" TO COMMENT OUT FIELDS
df = df[[ 'tuning_group',
#          'train_MSE', 
#          'test_MSE', 
         'train_RMSE', 
         'test_RMSE', 
#          'train_MAE', 
#          'test_MAE', 
#          'train_MAPE', 
#          'test_MAPE', 
#          'train_R2', 
#          'test_R2', 
         'scaler',
         'first_window', 
         'first_window_type', 
         'second_window',
         'second_window_type', 
#          'drop_nan_train', 
#          'min_periods', 
         'mod_loss',
         'mod_learning_rate', 
         'mod_n_estimators', 
         'mod_subsample',
#          'mod_min_samples_split',
#          'mod_min_samples_leaf', 
         'mod_max_depth',
         'mod_validation', 
         'num_features', 
         'train_samples_dropped'
        ]]
df.insert(3, "RMSE_diff", df["test_RMSE"] - df["train_RMSE"])
pd.set_option('display.max_rows', None)
#df = df.sort_values("test_RMSE").reset_index(drop = True)[:20]
df.sort_values("test_RMSE")[:10]

,tuning_group,train_RMSE,test_RMSE,RMSE_diff,scaler,first_window,first_window_type,second_window,second_window_type,mod_loss,mod_learning_rate,mod_n_estimators,mod_subsample,mod_max_depth,mod_validation,num_features,train_samples_dropped
34,1,19.685329,27.197247,7.511918,NaN,NaN,abs,NaN,avg,squared_error,0.05,20,1.0,7,NaN,14,0
14,1,19.912369,27.321562,7.409193,NaN,NaN,abs,NaN,avg,squared_error,0.01,100,1.0,7,NaN,14,0
13,1,20.978277,27.413536,6.435259,NaN,NaN,abs,NaN,avg,squared_error,0.01,100,1.0,5,NaN,14,0
57,1,20.540195,27.483474,6.943279,NaN,NaN,abs,NaN,avg,squared_error,0.10,10,1.0,5,NaN,14,0
33,1,20.786178,27.491830,6.705652,NaN,NaN,abs,NaN,avg,squared_error,0.05,20,1.0,5,NaN,14,0
58,1,19.424669,27.497188,8.072518,NaN,NaN,abs,NaN,avg,squared_error,0.10,10,1.0,7,NaN,14,0
15,1,18.536480,27.841821,9.305341,NaN,NaN,abs,NaN,avg,squared_error,0.01,100,1.0,9,NaN,14,0
56,1,21.557353,27.892715,6.335363,NaN,NaN,abs,NaN,avg,squared_error,0.10,10,1.0,3,NaN,14,0
59,1,18.023807,27.921441,9.897635,NaN,NaN,abs,NaN,avg,squared_error,0.10,10,1.0,9,NaN,14,0
12,1,21.940636,27.942006,6.001370,NaN,NaN,abs,NaN,avg,squared_error,0.01,100,1.0,3,NaN,14,0


# 4.0 Final Model

COMMENT REGARDING SMOOTHED DATA ANALYSIS: Since modeling results in the first tuning group were not promising, this analysis was ceased and no final model was determined. 